In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

# import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchmetrics as TM
# pl.utilities.seed.seed_everything(seed=42)
import numpy as np
import pandas as pd

import sys, os
source_path = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'preprocessing')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'metrics')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'predict')
sys.path.append(source_path)


import matplotlib.pyplot as plt

from dl import NeuralNetwork, Trainer
from preprocess import (
    show_df, 
    date_features, 
    preprocess, 
    ToTorch, 
    get_loader, 
    ts_split,
    cont_cat_split,
    dataloader_by_stock,
    get_data,
    dataloader_test_by_stock
)
# from metrics import calc_spread_return_sharpe
print(torch.__version__)
import pickle

import warnings
warnings.filterwarnings("ignore")

from dl import plot_loss


from predict import run_pred_step


KeyboardInterrupt: 

In [ ]:
print(torch.version.cuda), torch.cuda.is_available(), torch.cuda.device_count()

##  Get Data and train a Neural Network

# Get the unique security codes

In [ ]:
train_df = get_data()
# print('Unique adjustment factor:', train_df['AdjustmentFactor'].unique())
print(train_df['AdjustmentFactor'])
display(train_df.head())
train_df.info()

# Create model

### Args and constants
* Adding financials wes cumbersome maybe not worth now to add that data


In [ ]:

"""
SUBTRACT:
*) 3 FROM CONTINUOUS COLS BECAUSE OF POOLING
*) 1 FROM CAT_FEATURES TO MAKE MATRICES MATCH AFTER TORCH.CAT
"""
CONT_COLS=['Close', 'Open', 'High', 'Low', 'MarketCapitalization',         
           # 'NetSales', 'EquityToAssetRatio', 'TotalAssets', 'Profit', 
           # 'OperatingProfit', 'EarningsPerShare', 'Equity', 
           # 'BookValuePerShare', 'ResultDividendPerShare1stQuarter', 
           # 'ResultDividendPerShare2ndQuarter', 'ResultDividendPerShare3rdQuarter',
           # 'ResultDividendPerShareFiscalYearEnd', 'ResultDividendPerShareAnnual'
          ]
TS_IN_FEATURES = len(CONT_COLS)
print(TS_IN_FEATURES)
CAT_FEATURES = 4 + 4 - 1 # TEXT_COLS = ['Section/Products', '33SectorName', '17SectorName', 'Universe0']
print('CAT_FEATURES:', CAT_FEATURES)
EMBEDDING_DIM = 300
NO_EMBEDDING = 2000 #2 * len(df_train_cat)

In [ ]:

def create_model():
    model = None
    model = NeuralNetwork(
        in_features=TS_IN_FEATURES - 3, 
        units=128,
        out_features=1, 
        categorical_dim=CAT_FEATURES,
        no_embedding=NO_EMBEDDING, 
        emb_dim=EMBEDDING_DIM,
        n_blocks=4,
        n_stacks=1,
        dropout=0.3,
        pooling_sizes=3
    )

    print(model)
    return model

# Loop throug each stock
Create Trainer only once

In [ ]:
FOLDER = 'example_test_files'
BATCH_SIZE = 512

df = get_data(folder=FOLDER)

In [ ]:
%%time
stocks = train_df['SecuritiesCode'].unique()
count = 0
BATCH_SIZE = 512
weight_decay = 0.1
EPOCHS = 5

scaler_dict = {}

model = create_model()

trainer = Trainer(
    model, 
    optimizer_name='rmsprop', 
    lr=1.3333e-5, 
    weight_decay=weight_decay
)

torch.autograd.set_detect_anomaly(True)

train_loss_list = []
train_mae_list = []
valid_loss_list = []
valid_mae_list = []

for no_stock, stock in enumerate(stocks[: 5]):
    # try:
    train_loader, val_dataloader = None, None

    print(f'Stock-iteratation: {no_stock}')
    print(f'Start training for stock: {stock}')

    train_dataloader, val_dataloader = dataloader_by_stock(
        train_df, 
        stock, 
        batch_size=BATCH_SIZE, 
        continous_cols=CONT_COLS,
        return_scaler=False
    )
    # if count > 1:
    #     EPOCHS = 15
    train_loss, train_mae, val_loss, val_mae = trainer.fit_epochs(
        train_dataloader, 
        val_dataloader, 
        use_cyclic_lr=True, 
        x_cat=True, 
        epochs=EPOCHS
    )
    plot_loss(train_loss, val_loss, f'Train-loss: {stock}', f'Valid-loss: {stock}')
    plot_loss(train_mae, val_mae, f'Train-mae: {stock}', f'Valid-mae: {stock}')
    # train_loss_list.extend(train_loss)
    # train_mae_list.extend(train_mae)
    # valid_loss_list.extend(val_loss)
    # valid_mae_list.extend(val_mae)
    # scaler_dict[stock] = scaler
    print('#' * 20)
    print()
    count += 1
    
    
    # except Exception as e:
    #     print(f'Training loop: {e}')
    
torch.save(model.state_dict(), './trained_model.pt')
with open('scaler_dict.pkl', 'wb') as f:
    pickle.dump(scaler_dict, f)

In [ ]:
# from dl import plot_loss
# plt.plot(train_loss_list)
# plt.title('Train loss');
# plt.show()

# plt.plot(valid_loss_list)
# plt.title('Valid loss')
# plt.show()

# plot_loss(train_loss_list, valid_loss_list, 'Train-loss', 'Valid-loss')

In [ ]:
train_loss_list[:5]

In [ ]:
FOLDER = 'example_test_files'
BATCH_SIZE = 512

df = get_data(folder=FOLDER)

In [ ]:


for stock in stocks:
    # teast_dataloader = None
    # if count > 500:
    #     break
    # try:
    print(f'Run prediction for stock: {stock}')
    # transformer = scaler_dict[stock]
    test_dataloader = dataloader_test_by_stock(
        df, 
        stock, 
        # transformer=transformer,
        batch_size=BATCH_SIZE, 
        continous_cols=CONT_COLS,
        target_col=None
    )
    for batch, data in enumerate(test_dataloader):
        model.to('cpu')
        x_con_test = data['num_features'].to("cpu")
        x_cat_test = data['cat_features'].to('cpu')
        print('x.shape:', x_con_test.shape)
        print('x_cat.shape:', x_cat_test.t().shape)
        pred = model(x_con_test, x_cat_test)
        print('pred:', pred)
        print()

In [ ]:
pred_df = pd.DataFrame()

for stock in stocks:
    # teast_dataloader = None
    # if count > 500:
    #     break
    # try:
    print(f'Run prediction for stock: {stock}')
    # transformer = scaler_dict[stock]
    test_dataloader = dataloader_test_by_stock(
        df, 
        stock, 
        # transformer=transformer,
        batch_size=BATCH_SIZE, 
        continous_cols=CONT_COLS,
        target_col=None
    )
    pred_list = run_pred_step(test_dataloader, model, x_cat=True, target=False)
    df_pred = pd.DataFrame(pred_list, index=[stock])
    pred_df = pd.concat([pred_df, df_pred])#.pivot()
    print('pred_df')
    print(pred_df[:5])
    # pred_df[stock] = pred_list
    print()
    count += 1
#     except Exception as e:
#         print(f'Exception {e}')
        

# pred_df = pd.DataFrame.from_dict(pred_df)


### 